### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.simplefilter("ignore")

pd.options.mode.chained_assignment = None

### Import Excel Files

In [ ]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\Semi-Monthly Closed Invoice\2022 Semi-Monthly Closed Invoice\2022 Semi-Monthly Report\Semi-Monthly Report Others Count'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

In [ ]:
df1 = excel_list[0]['Sheet1'].drop('Count', axis = 1).sort_values(['Accounting_CustomerID'])
df2 = excel_list[1]['Sheet1'].drop('Count', axis = 1).sort_values(['Accounting_CustomerID'])
df3 = excel_list[2]['Sheet1'].drop('Count', axis = 1).sort_values(['Accounting_CustomerID'])
# df4 = excel_list[3]['Sheet1'].drop('Count', axis = 1).sort_values(['Accounting_CustomerID'])

In [ ]:
df_cb1 = pd.merge(df1, df2, how = 'outer',  on = ['Accounting_CustomerID', 'CustomerID', 
                                                 'WISE Facility', 'Bill Pay Category', 'Bill To']) 
df_cb2 = pd.merge(df_cb1, df3, how = 'outer',  on = ['Accounting_CustomerID', 'CustomerID', 
                                                 'WISE Facility', 'Bill Pay Category', 'Bill To']) 
# df_cb3 = pd.merge(df_cb2, df4, how = 'outer',  on = ['Accounting_CustomerID', 'CustomerID', 
#                                                  'Facility', 'Bill Pay Category', 'Bill To']) 

### Generate Semi-Monthly Report with Billing Count and without Invoice Number

In [ ]:
df_count = df_cb2.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).nunique(axis = 1)
df_count

In [ ]:
df_count.columns.values[[range(5, len(df_count.columns)-1)]] = np.repeat('Billing Period Start', len(range(5, len(df_count.columns)-1)))

In [ ]:
df_count

In [ ]:
# month = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

month = np.array(['Jan', 'Feb', 'Mar'])
    
semi_month = np.array([])
for m in month:
    semi_month = np.append(semi_month, m + '_Mid')
    semi_month = np.append(semi_month, m + '_End')

bp = np.where(df_count.columns.values == 'Billing Period Start')
# dn = np.where(df_output.columns.values == 'DocNo')

# dn_month = [x + '_DocNo' for x in semi_month]

df_count.columns.values[bp] = semi_month
# df_output.columns.values[dn] = dn_month

df_count

In [ ]:
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()
df_count = df_count.drop_duplicates(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count

### Export Semi-Monthly Report As Excel File

In [ ]:
df_count.to_excel("2022 Semi-Monthly Report Others Count.xlsx", index = False, header = True)